<a href="https://colab.research.google.com/github/acmilannesta/Bert-embedding/blob/master/GoogleQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!nvidia-smi

Tue Feb  4 00:12:16 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.48.02    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
# nlp aug
!pip install transformers

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%tensorflow_version 2.x
import sys
sys.path.extend(['/content/drive/My Drive/GoogleQA'])
import numpy as np
import pandas as pd
import os, gc, codecs
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold, GroupKFold, KFold, RepeatedKFold
from scipy.stats import spearmanr
from tqdm import tqdm
import tensorflow_hub as hub
from warmup_v2 import AdamWarmup, calc_train_steps
tf.get_logger().setLevel('ERROR') 
from transformers import AlbertTokenizer, TFAlbertModel
import warnings
warnings.filterwarnings('ignore')
print(tf.__version__)

TensorFlow 2.x selected.
2.1.0


##Load dataset

In [0]:
data_path = 'drive/My Drive/GoogleQA/Data/'
train = pd.read_csv(data_path+'train.csv')
test = pd.read_csv(data_path+'test.csv')
target_col = train.columns.tolist()[11:42]
q_col, a_col = target_col[:21], target_col[21:]


# Label encoding category and host
class MyLabelEncoder(object):
    """safely handle unknown label"""
    def __init__(self):
        self.mapper = {}
    def fit(self, X):
        uniq_X = np.unique(X)
        # reserve 0 for unknown
        self.mapper = dict(zip(uniq_X, range(1, len(uniq_X) + 1)))
        return self
    def fit_transform(self, X):
        self.fit(X)
        return self.transform(X)
    def _map(self, x):
        return self.mapper.get(x, 0)
    def transform(self, X):
        return list(map(self._map, X))

category_encoder = MyLabelEncoder().fit(train['category'])
train['category'] = category_encoder.transform(train['category'])
test['category'] = category_encoder.transform(test['category'])
host_encoder = MyLabelEncoder().fit(train['host'])
train['host'] = host_encoder.transform(train['host'])
test['host'] = host_encoder.transform(test['host'])

# average question targets within the same questions
def q_avg(data_df):
    q_col_med = data_df.groupby(['question_title'], as_index=False)[q_col].mean()
    return data_df.drop(target_col, 1).merge(q_col_med, how='left', on='question_title').drop_duplicates('question_title').reset_index(drop=True)

# assign class weights
def label_wt(data_df, cols, wt1=2, wt2=1.5, wt3=1):
    for col in cols:
        wt = data_df[col].value_counts(normalize=True).rename(col+'_wt').to_frame()
        wt[col+'_wt'] = np.where(wt[col+'_wt']<=0.01, wt1, np.where(wt[col+'_wt']<=0.05, wt2, wt3))
        data_df = data_df.merge(wt, 'left', left_on=col, right_on=wt.index)
    return data_df

# train, train_q, train_a = label_wt(train, target_col), label_wt(train_q, q_col), label_wt(train_a, a_col)
col_wt, q_col_wt, a_col_wt = [x+'_wt' for x in target_col], [x+'_wt' for x in q_col], [x+'_wt' for x in a_col]

## Parameter setting

In [0]:
MAXLEN = 512 
BATCH_SIZE = 4
NUM_EPOCHS = 4
NUM_CLASSES = 30
NUM_AUX = 6
LR = 5e-5
MIN_LR = 0

model_path = 'drive/My Drive/GoogleQA/Albert'
save_path = 'Albert'
save_model = 'Albert' 
if save_path not in os.listdir('drive/My Drive/GoogleQA/Models'):
    os.system(f'mkdir drive/My\ Drive/GoogleQA/Models/{save_path}') 

CASED = False


In [0]:
def _trim_input(title, question, answer, max_sequence_length=MAXLEN,
                t_max_len=15, q_max_len=245, a_max_len=248):
    t_len = len(title)
    q_len = len(question)
    a_len = len(answer)
    if (t_len + q_len + a_len +6) > max_sequence_length:
        if t_max_len > t_len:
            t_new_len = t_len
            a_max_len = a_max_len + np.floor((t_max_len - t_len) / 2)
            q_max_len = q_max_len + np.ceil((t_max_len - t_len) / 2)
        else:
            t_new_len = t_max_len
        if a_max_len > a_len:
            a_new_len = a_len
            q_new_len = q_max_len + (a_max_len - a_len)
        elif q_max_len > q_len:
            a_new_len = a_max_len + (q_max_len - q_len)
            q_new_len = q_len
        else:
            a_new_len = a_max_len
            q_new_len = q_max_len
        if t_new_len + a_new_len + q_new_len +6 != max_sequence_length:
            raise ValueError("New sequence length should be %d, but is %d"
                    % (max_sequence_length, (t_new_len + a_new_len + q_new_len +6)))
        head_t_new_len = int(0.25 * t_new_len)
        tail_t_new_len = int(t_new_len - head_t_new_len)

        head_q_new_len = int(0.25 * q_new_len)
        tail_q_new_len = int(q_new_len - head_q_new_len)

        head_a_new_len = int(0.25 * a_new_len)
        tail_a_new_len = int(a_new_len - head_a_new_len)

        title = title[:head_t_new_len] + title[-tail_t_new_len:]
        question = question[:head_q_new_len] + question[-tail_q_new_len:]
        answer = answer[:head_a_new_len] + answer[-tail_a_new_len:]

    return title, question, answer



# for category in np.unique(train['category']):
#     tokenizer.vocab[f'[{category}]'] = len(tokenizer.vocab)
# for host in np.unique(train['host']):
#     tokenizer.vocab[f'[{host}]'] = len(tokenizer.vocab)
# tokenizer.vocab['[category_unk]'] = len(tokenizer.vocab)
# tokenizer.vocab['[host_unk]'] = len(tokenizer.vocab)

def q_trim(tokens, max_length=MAXLEN):
# def q_trim(q_, a_, max_length=MAXLEN):
    if len(tokens) > max_length:
        head_length = int(0.25 * max_length)
        tail_length = max_length - head_length
        tokens = tokens[:head_length] + tokens[-tail_length:]
    return tokens
    # if len(q_) >= int(max_length*0.7):
    #     head_length = int(0.25 * int(max_length*0.7))
    #     tail_length = int(max_length*0.7) - head_length
    #     q_ = q_[:head_length] + q_[-tail_length:]

    # if len(q_) + len(a_) > max_length:
    #     a_length = max_length - len(q_)
    #     head_a_length = int(0.25*a_length)
    #     tail_a_length = a_length - head_a_length
    #     a_ = a_[:head_a_length] + a_[-tail_a_length:]
    # return q_, a_

def a_trim(q_, a_, max_length=MAXLEN):
    a_max_length = int(max_length*0.75)
    if len(a_) >= a_max_length:
        head_length = int(0.25 * a_max_length)
        tail_length = a_max_length - head_length
        a_ = a_[:head_length] + a_[-tail_length:]

    if len(q_) + len(a_) > max_length:
        q_length = max_length - len(a_)
        head_q_length = int(0.25*q_length)
        tail_q_length = q_length - head_q_length
        q_ = q_[:head_q_length] + q_[-tail_q_length:]
    return q_, a_

## Roberta Model

### Roberta tokenizer

In [0]:
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')

def convert_roberta(data_df, model, targets, branch='training'):
    data_df = data_df.reset_index(drop=True)
    global tokenizer
    global MAXLEN
    global target_col
    ids = []
    for i in tqdm(range(len(data_df))):
        a = tokenizer.tokenize(data_df.loc[i, 'question_title'])
        b = tokenizer.tokenize(data_df.loc[i, 'question_body'])
        c = tokenizer.tokenize(data_df.loc[i, 'answer'])
        if model == 'qa':
            a, b, c = _trim_input(a, b, c, t_max_len=15, q_max_len=244, a_max_len=247)
            question = tokenizer.encode(a+b)
            answer = tokenizer.encode(c)
            ids.append(question+answer)
        if model == 'q':
            question = tokenizer.encode(a+b)
            # answer = tokenizer.convert_tokens_to_ids(['[AN]']+c+['[SEP]'])
            question_trim = q_trim(question)
            ids.append(question_trim)
        if model == 'a':
            question = tokenizer.encode(a+b)
            answer = tokenizer.encode(c)      
            question_trim, answer_trim = a_trim(question, answer)
            ids.append(question_trim + answer_trim)
    aux = data_df[['category', 'host']]    
    if branch == 'training':
        targets = data_df[targets]
        return ids, np.array(aux, dtype='int32'), np.array(targets)
        # return [ids, segments], np.array(targets)
    else:
        return ids, np.array(aux, dtype='int32')
        # return [ids, segments]

### Roberta generator

In [0]:
"""## Data Generator"""
def seq_padding(X, padding=0):
    L = [len(x) for x in X]
    ML = min(max(L), MAXLEN)
    # ML = MAXLEN
    out = np.array([np.concatenate([x, [padding] * (ML - len(x))]) if len(x) < ML else x[:ML] for x in X])
    return tf.convert_to_tensor(out, dtype=tf.int32)
    
class roberta_generator:
    def __init__(self, data, batch_size=BATCH_SIZE, branch='train', multi_target=True):
        self.data = data
        self.batch_size = batch_size
        self.branch = branch
        self.steps = len(self.data) // self.batch_size
        self.multi_target=multi_target
        if len(self.data) % self.batch_size != 0:
            self.steps += 1

    def __len__(self):
        return self.steps

    def __iter__(self):
        while True:
            if self.branch == 'train':
                np.random.shuffle(self.data)
            for i in range(self.steps):
                d = self.data[i * self.batch_size: (i + 1) * self.batch_size]
                X1 = seq_padding([x[0] for x in d])
                aux = np.array([x[1][0] for x in d]).reshape(-1, 1)
                aux = tf.convert_to_tensor(aux, dtype=tf.int32)
                aux2 = np.array([x[1][1] for x in d]).reshape(-1, 1)
                aux2 = tf.convert_to_tensor(aux2, dtype=tf.int32)
                if self.branch != 'train':
                    yield [X1, aux, aux2]
                else:                    
                    if self.multi_target:
                        Y = []
                        num_targets = int(d[0][2].shape[0] / 2)
                        for i in range(num_targets):
                            Y.append(tf.convert_to_tensor([x[2][[i, i+num_targets]] for x in d], dtype=tf.float32))
                    else:
                        Y = tf.convert_to_tensor([x[2] for x in d], dtype=tf.float32)
                    yield [X1, aux, aux2], Y


### Roberta model

In [0]:
def custom_loss(y_true, y_pred):
    return keras.metrics.binary_crossentropy(tf.reshape(y_true[:,0],(-1,1)), y_pred) * y_true[:,1]

def roberta_build(len_train, num_targets, dropout=0.2, lr=LR, epochs=NUM_EPOCHS, roberta_trainable=True, multi_target=True):
 
    input_text = keras.layers.Input((None,), dtype=tf.int32, name='input_text')
    input_category = keras.layers.Input((1,), dtype=tf.int32, name='input_category')
    input_host = tf.keras.layers.Input((1,), dtype=tf.int32, name='input_host')

    roberta_model = TFAlbertModel.from_pretrained(model_path, trainable=roberta_trainable)
    q_outputs = roberta_model(input_text)[0]
    q_outputs1 = keras.layers.GlobalAveragePooling1D()(q_outputs)
    q_outputs1 = keras.layers.Dropout(dropout)(q_outputs1)

    category_emb = keras.layers.Embedding(input_dim=len(category_encoder.mapper)+1, output_dim=32)(input_category)
    category_emb = keras.layers.SpatialDropout1D(0.1)(category_emb)

    host_emb = keras.layers.Embedding(input_dim=len(host_encoder.mapper)+2, output_dim=32)(input_host)
    host_emb = keras.layers.SpatialDropout1D(0.1)(host_emb)

    features_dense = keras.layers.concatenate([category_emb, host_emb], axis=1)
    features_dense = keras.layers.Flatten()(features_dense)

    dense = keras.layers.concatenate([q_outputs1, features_dense])

    if multi_target:
        # outputs = [keras.layers.Lambda(transfer)(keras.layers.Dense(1, activation='sigmoid')(dense)) for _ in range(num_targets)]
        outputs = [keras.layers.Dense(1, activation='sigmoid')(dense) for _ in range(num_targets)]
        loss_func = [custom_loss]*num_targets

    else:
        # outputs = keras.layers.Lambda(transfer)(keras.layers.Dense(num_targets, activation='sigmoid')(dense))
        outputs = keras.layers.Dense(num_targets, activation='sigmoid')(dense)
        loss_func = ['binary_crossentropy']
    # model = keras.Model([q_in, q2_in, q3_in], outputs)
    model = keras.Model([input_text, input_category, input_host], outputs)


    decay_steps, warmup_steps = calc_train_steps(
        len_train,
        batch_size=BATCH_SIZE,
        epochs=epochs,
    )

    model.compile(
        loss=loss_func,
        optimizer=AdamWarmup(
            decay_steps=decay_steps,
            warmup_steps=warmup_steps,
            lr=lr,
            min_lr=MIN_LR,
            ),
        # optimizer = optimizer_lamb.LAMB(learning_rate=lr)
        )


    return model

### Roberta callbacks

In [0]:
# kf = RepeatedKFold(n_splits=10, n_repeats=1, random_state=1627)
def compute_spearmanr(trues, preds):
    rhos = []
    for col_trues, col_pred in zip(trues.T, preds.T):
        rhos.append(spearmanr(col_trues, col_pred).correlation)
    # rho_df = pd.DataFrame(dict(col=cols, rhos=rhos))
    return np.nanmean(rhos)#, rho_df

class IntervalEval(keras.callbacks.Callback):
    def __init__(self, save_model, rho_init=-1, multi_target=True):
        global NUM_EPOCHS
        super().__init__()
        self.rho = []
        self.model_weights = []
        self.save_model = save_model
        self.multi_target = multi_target
        self.rho_init = rho_init
    def on_epoch_end(self, epoch, logs={}):
        val_pred = self.model.predict_generator(valid_D.__iter__(), len(valid_D), verbose=1)
        if self.multi_target:
            val_pred = np.array(val_pred).squeeze().T
        nunique = [i for i in range(val_pred.shape[1]) if len(np.unique(val_pred[:, i]))==1]
        if len(nunique)>0:
            print(f'{len(nunique)} cols with monotonous preds!')
        score = compute_spearmanr(val_pred, val_y)
        print('Spearman - {:.5f}'.format(score))
        if score>self.rho_init:
            self.rho_init = score
            print('Save Weights')
            self.model.save_weights('drive/My Drive/GoogleQA/Models/{:}/{:}-{:}.h5'.format(save_path, self.save_model, i))
        self.rho.append(score)
        self.model_weights.append(self.model.get_weights())

    def on_train_end(self, logs=None):
        """
        Weighted average of weights from two best epochs

        """
        rho_selected = np.argsort(self.rho)[-2:]
        print(f'Best epoch {rho_selected[1]+1} and second best epoch {rho_selected[0]+1} selected')
        swa_weights = [x * 0.3 + y * 0.7 for x, y in zip(self.model_weights[rho_selected[0]], self.model_weights[rho_selected[1]])]
        self.model.set_weights(swa_weights)
        val_pred = self.model.predict_generator(valid_D.__iter__(), len(valid_D), verbose=1)
        if self.multi_target:
            val_pred = np.array(val_pred).squeeze().T
        nunique = [i for i in range(val_pred.shape[1]) if len(np.unique(val_pred[:, i]))==1]
        if len(nunique)>0:
            print(f'{len(nunique)} cols with monotonous preds!')
        score = compute_spearmanr(val_pred, val_y)
        print('SWD Model Spearman - {:.5f}'.format(score))
        if score>self.rho_init:
            self.rho_init = score
            print('Save SWD Weights')
            self.model.save_weights('drive/My Drive/GoogleQA/Models/{:}/{:}-{:}.h5'.format(save_path, self.save_model, i))
        del self.rho, self.model_weights, swa_weights
        gc.collect()



### Roberta Q model

In [0]:
kf = RepeatedKFold(n_splits=5, n_repeats=1, random_state=0)
idx_q = [x for x in kf.split(train, groups=train.question_title)]
multi_target = True

for i, (tr_idx, val_idx) in enumerate(idx_q[1:], 2):
    keras.backend.clear_session()
    gc.collect()

    print('\nFold - {:}\n'.format(i))
    tr, val = train.loc[tr_idx], train.loc[val_idx]
    tr = q_avg(tr)
    tr = label_wt(tr, q_col)

    tr_x, tr_aux, tr_y = convert_roberta(tr, model='q', targets=q_col+q_col_wt)
    val_x, val_aux, val_y = convert_roberta(val, model='q', targets=q_col)

    if sum(val[q_col].nunique()==1)>0:
        print(f'\n{sum(val[q_col].nunique()==1)} with Monotonous Label Detected!\n')

    train_D = roberta_generator(list(zip(tr_x, tr_aux, tr_y)), multi_target=multi_target)
    valid_D = roberta_generator(list(zip(val_x, val_aux)), branch='valid', multi_target=multi_target)
    
    # stage 1 fine tunning
    ieval = IntervalEval(save_model=save_model+'_q', multi_target=multi_target)
    model = roberta_build(
        len_train=len(tr), 
        num_targets=len(q_col), 
        multi_target=multi_target,
        dropout=0.1,
        )
    model.fit_generator(
        train_D.__iter__(),
        steps_per_epoch=len(train_D),
        epochs=NUM_EPOCHS,
        callbacks = [ieval]
    )   
    rho_stage1 = ieval.rho_init

    # stage 2 fine tunning
    ieval = IntervalEval(save_model=save_model+'_q', multi_target=multi_target, rho_init=rho_stage1)
    model = roberta_build(
        len_train=len(tr), 
        num_targets=len(q_col), 
        lr=1e-4, 
        roberta_trainable=False, 
        multi_target=multi_target,
        dropout=0.1,
        )
    model.load_weights('drive/My Drive/GoogleQA/Models/{:}/{:}-{:}.h5'.format(save_path, save_model+'_q', i))

    model.fit_generator(
        train_D.__iter__(),
        steps_per_epoch=len(train_D),
        epochs=4,
        callbacks = [ieval]
    )

    gc.collect()

  0%|          | 0/3163 [00:00<?, ?it/s]


Fold - 2



100%|██████████| 1216/1216 [00:01<00:00, 648.77it/s]


Train for 791 steps
Epoch 1/4
304/304 [==============================] - 38s 125ms/step
Spearman - 0.38360
Save Weights
791/791 [==============================] - 442s 558ms/step - loss: 11.2110 - dense_loss: 0.4538 - dense_1_loss: 0.8950 - dense_2_loss: 0.3269 - dense_3_loss: 0.7561 - dense_4_loss: 0.5998 - dense_5_loss: 0.5787 - dense_6_loss: 0.8250 - dense_7_loss: 0.8500 - dense_8_loss: 0.6001 - dense_9_loss: 0.1111 - dense_10_loss: 0.8290 - dense_11_loss: 0.7726 - dense_12_loss: 0.2684 - dense_13_loss: 0.1230 - dense_14_loss: 0.1926 - dense_15_loss: 0.3282 - dense_16_loss: 0.6416 - dense_17_loss: 0.5416 - dense_18_loss: 0.7290 - dense_19_loss: 0.0329 - dense_20_loss: 0.7556
Epoch 2/4
304/304 [==============================] - 36s 120ms/step
Spearman - 0.42096
Save Weights
791/791 [==============================] - 403s 510ms/step - loss: 9.2117 - dense_loss: 0.4176 - dense_1_loss: 0.7711 - dense_2_loss: 0.2145 - dense_3_loss: 0.6506 - dense_4_loss: 0.5493 - dense_5_loss: 0.4774 - d

  0%|          | 0/3180 [00:00<?, ?it/s]

...

Train for 792 steps
Epoch 1/4
304/304 [==============================] - 39s 128ms/step
Spearman - 0.38371
Save Weights
792/792 [==============================] - 444s 561ms/step - loss: 11.2768 - dense_loss: 0.4530 - dense_1_loss: 0.8526 - dense_2_loss: 0.2845 - dense_3_loss: 0.7049 - dense_4_loss: 0.6017 - dense_5_loss: 0.6314 - dense_6_loss: 0.8188 - dense_7_loss: 0.9136 - dense_8_loss: 0.6047 - dense_9_loss: 0.0692 - dense_10_loss: 0.8809 - dense_11_loss: 0.6739 - dense_12_loss: 0.2181 - dense_13_loss: 0.2786 - dense_14_loss: 0.1780 - dense_15_loss: 0.3185 - dense_16_loss: 0.6922 - dense_17_loss: 0.5510 - dense_18_loss: 0.8012 - dense_19_loss: 0.0768 - dense_20_loss: 0.6731
Epoch 2/4
304/304 [==============================] - 37s 122ms/step
Spearman - 0.42716
Save Weights
792/792 [==============================] - 404s 511ms/step - loss: 9.2330 - dense_loss: 0.4199 - dense_1_loss: 0.7756 - dense_2_loss: 0.2123 - dense_3_loss: 0.6554 - dense_4_loss: 0.5544 - dense_5_loss: 0.5111 - d

  0%|          | 0/3186 [00:00<?, ?it/s]


Fold - 5



100%|██████████| 1215/1215 [00:01<00:00, 665.44it/s]


Train for 797 steps
Epoch 1/4
304/304 [==============================] - 37s 122ms/step
Spearman - 0.37541
Save Weights
797/797 [==============================] - 445s 559ms/step - loss: 11.3719 - dense_loss: 0.5945 - dense_1_loss: 0.7970 - dense_2_loss: 0.3563 - dense_3_loss: 0.7076 - dense_4_loss: 0.6309 - dense_5_loss: 0.6832 - dense_6_loss: 0.8348 - dense_7_loss: 0.8329 - dense_8_loss: 0.6356 - dense_9_loss: 0.0689 - dense_10_loss: 0.8199 - dense_11_loss: 0.6960 - dense_12_loss: 0.2616 - dense_13_loss: 0.2492 - dense_14_loss: 0.1905 - dense_15_loss: 0.3260 - dense_16_loss: 0.7091 - dense_17_loss: 0.5845 - dense_18_loss: 0.6639 - dense_19_loss: 0.0441 - dense_20_loss: 0.6854
Epoch 2/4
304/304 [==============================] - 35s 117ms/step
Spearman - 0.41982
Save Weights
797/797 [==============================] - 409s 514ms/step - loss: 9.2489 - dense_loss: 0.4212 - dense_1_loss: 0.7635 - dense_2_loss: 0.2144 - dense_3_loss: 0.6635 - dense_4_loss: 0.5512 - dense_5_loss: 0.4883 - d

### Roberta A model

In [0]:
kf = RepeatedKFold(n_splits=5, n_repeats=1, random_state=0)
idx_a = [x for x in kf.split(train, groups=train.question_title)]
multi_target = True

for i, (tr_idx, val_idx) in enumerate(idx_a[3:4], 4):
    keras.backend.clear_session()
    gc.collect()

    print('\nFold - {:}\n'.format(i))
    tr, val = train.loc[tr_idx], train.loc[val_idx]
    tr = label_wt(tr, a_col)

    tr_x, tr_aux, tr_y = convert_roberta(tr, model='a', targets=a_col+a_col_wt)
    val_x, val_aux, val_y = convert_roberta(val, model='a', targets=a_col)

    if sum(val[q_col].nunique()==1)>0:
        print(f'\n{sum(val[q_col].nunique()==1)} with Monotonous Label Detected!\n')

    train_D = roberta_generator(list(zip(tr_x, tr_aux, tr_y)), multi_target=multi_target)
    valid_D = roberta_generator(list(zip(val_x, val_aux)), branch='valid', multi_target=multi_target)
    
    # stage 1 fine tunning
    ieval = IntervalEval(save_model=save_model+'_a', multi_target=multi_target)
    model = roberta_build(
        len_train=len(tr), 
        num_targets=len(a_col), 
        multi_target=multi_target,
        dropout=0.1,
        )
    model.fit_generator(
        train_D.__iter__(),
        steps_per_epoch=len(train_D),
        epochs=NUM_EPOCHS,
        callbacks = [ieval]
    )   
    rho_stage1 = ieval.rho_init

    # stage 2 fine tunning
    ieval = IntervalEval(save_model=save_model+'_a', multi_target=multi_target, rho_init=rho_stage1)
    model = roberta_build(
        len_train=len(tr), 
        num_targets=len(a_col), 
        lr=1e-4, 
        roberta_trainable=False, 
        multi_target=multi_target,
        dropout=0.1,
        )
    model.load_weights('drive/My Drive/GoogleQA/Models/{:}/{:}-{:}.h5'.format(save_path, save_model+'_a', i))

    model.fit_generator(
        train_D.__iter__(),
        steps_per_epoch=len(train_D),
        epochs=4,
        callbacks = [ieval]
    )

    gc.collect()

  1%|          | 55/4863 [00:00<00:08, 538.15it/s]


Fold - 4



100%|██████████| 1216/1216 [00:02<00:00, 582.51it/s]


Train for 1216 steps
Epoch 1/4
304/304 [==============================] - 79s 261ms/step
Spearman - 0.32116
Save Weights
1216/1216 [==============================] - 1440s 1s/step - loss: 4.1047 - dense_loss: 0.3018 - dense_1_loss: 0.7373 - dense_2_loss: 0.2214 - dense_3_loss: 0.1868 - dense_4_loss: 0.4972 - dense_5_loss: 0.6074 - dense_6_loss: 0.4919 - dense_7_loss: 0.6395 - dense_8_loss: 0.4214
Epoch 2/4
304/304 [==============================] - 78s 255ms/step
Spearman - 0.33743
Save Weights
1216/1216 [==============================] - 1387s 1s/step - loss: 3.6100 - dense_loss: 0.2878 - dense_1_loss: 0.7001 - dense_2_loss: 0.2122 - dense_3_loss: 0.1724 - dense_4_loss: 0.4801 - dense_5_loss: 0.4665 - dense_6_loss: 0.4000 - dense_7_loss: 0.5441 - dense_8_loss: 0.3468
Epoch 3/4
304/304 [==============================] - 78s 256ms/step
Spearman - 0.34258
Save Weights
1216/1216 [==============================] - 1393s 1s/step - loss: 3.4394 - dense_loss: 0.2786 - dense_1_loss: 0.6943 - d

### oof pred

In [0]:
# oof-prediction
multi_target = True
model_q = roberta_build(len_train=len(train), num_targets=len(q_col), multi_target=multi_target)
model_a = roberta_build(len_train=len(train), num_targets=len(a_col), multi_target=multi_target)
oof_pred_q = train[['qa_id']+q_col].copy()
oof_pred_q[q_col] = 0
oof_pred_a = train[['qa_id']+a_col].copy()
oof_pred_a[a_col] = 0


kf = RepeatedKFold(n_splits=5, n_repeats=1, random_state=0)
idx_q = [x for x in kf.split(train, groups=train.question_title)]
for i, (tr_idx, val_idx) in enumerate(idx_q, 1):
    print('\nFold - {:}\n'.format(i))
    tr, val = train.loc[tr_idx], train.loc[val_idx]
    val_x, val_aux, val_y = convert_roberta(val, 'q', targets=q_col)
    valid_D = roberta_generator(list(zip(val_x, val_aux)), branch='valid', multi_target=multi_target)
    model_q.load_weights('drive/My Drive/GoogleQA/Models/{}/{}-{}.h5'.format(save_path, save_model+'_q', i))
    val_pred_q = model_q.predict_generator(valid_D.__iter__(), len(valid_D), verbose=1)
    oof_pred_q.loc[val_idx, q_col] += np.array(val_pred_q).squeeze().T# / 2
    score = compute_spearmanr(val_y, np.array(val_pred_q).squeeze().T)
    print('\nQ - Spearman - {:.5f}\n'.format(score))

idx_a = [x for x in kf.split(train, groups=train.question_title)]
for i, (tr_idx, val_idx) in enumerate(idx_a, 1):
    print('\nFold - {:}\n'.format(i))
    tr, val = train.loc[tr_idx], train.loc[val_idx]
    val_x, val_aux, val_y = convert_roberta(val, 'a', targets=a_col)
    valid_D = roberta_generator(list(zip(val_x, val_aux)), branch='valid', multi_target=multi_target)
    model_a.load_weights('drive/My Drive/GoogleQA/Models/{}/{}-{}.h5'.format(save_path, save_model+'_a', i))
    val_pred_a = model_a.predict_generator(valid_D.__iter__(), len(valid_D), verbose=1)
    oof_pred_a.loc[val_idx, a_col] += np.array(val_pred_a).squeeze().T
    score = compute_spearmanr(val_y, np.array(val_pred_a).squeeze().T)
    print('\nA - Spearman - {:.5f}\n'.format(score))

    gc.collect()

oof_pred_qa_sep = pd.concat([oof_pred_q, oof_pred_a[a_col]], axis=1)
# oof_pred = oof_pred_qa[target_col].values * 0.4 + np.concatenate((oof_pred_q[q_col].values, oof_pred_a[a_col].values), axis=1) * 0.6
oof_pred_qa_sep.to_csv('drive/My Drive/GoogleQA/Data/train_oof_pred_roberta.csv', index=False)